In [1]:
%%capture
!pip install pyloudnorm
!pip install pysndfx
!pip install speechbrain
!pip install transformers
!pip install speechbrain.pretrained
!pip install torchmetrics

In [2]:
import os
from multiprocessing import Process
import subprocess
import glob

import random
import warnings
import tarfile

import numpy as np
import pandas as pd
import pyloudnorm as pyln
import soundfile as sf
import tqdm as tqdm_module
import zipfile

import tqdm.contrib.concurrent
import functools
from pysndfx import AudioEffectsChain

import librosa

import torchaudio
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
import torchaudio.transforms as T
import torchmetrics

from IPython.display import Audio

import speechbrain as sb
from speechbrain.dataio.dataio import read_audio
from IPython.display import Audio
from speechbrain.inference.separation import SepformerSeparation as separator

from torchmetrics.audio import ScaleInvariantSignalNoiseRatio
from torchmetrics.audio import ScaleInvariantSignalDistortionRatio

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# !unzip /content/drive/MyDrive/LibriMix.zip

Streaming output truncated to the last 5000 lines.
  inflating: wav8k/max/test/mix_both/1995-1826-0023_8463-294828-0020.wav  
  inflating: wav8k/max/test/mix_both/5142-36377-0022_4992-23283-0019.wav  
  inflating: wav8k/max/test/mix_both/4446-2271-0004_3570-5694-0002.wav  
  inflating: wav8k/max/test/mix_both/7021-79740-0000_1320-122612-0010.wav  
  inflating: wav8k/max/test/mix_both/2830-3980-0031_1995-1837-0003.wav  
  inflating: wav8k/max/test/mix_both/260-123286-0005_1580-141084-0026.wav  
  inflating: wav8k/max/test/mix_both/6829-68769-0049_260-123288-0017.wav  
  inflating: wav8k/max/test/mix_both/237-134493-0011_7176-92135-0024.wav  
  inflating: wav8k/max/test/mix_both/2300-131720-0040_61-70970-0027.wav  
  inflating: wav8k/max/test/mix_both/4507-16021-0024_5639-40744-0038.wav  
  inflating: wav8k/max/test/mix_both/7127-75946-0018_2300-131720-0008.wav  
  inflating: wav8k/max/test/mix_both/1995-1837-0025_5142-33396-0014.wav  
  inflating: wav8k/max/test/mix_both/4446-2273-0008_

In [ ]:
!mkdir LibriSpeech
!mkdir LibriMix

In [ ]:
storage_dir = "/content"
librispeech_dir = os.path.join(storage_dir,"LibriSpeech")
librimix_outdir = storage_dir
wham_dir = os.path.join(storage_dir,"wham_noise")

In [ ]:
def LibriSpeech_test_clean():
  if not os.path.exists(os.path.join(storage_dir ,"test_clean")):
    print("Download LibriSpeech/test_clean into",storage_dir )
    subprocess.run(["wget","-c","--tries=0","--read-timeout=20","http://www.openslr.org/resources/12/test-clean.tar.gz","-P",storage_dir ])
    # subprocess.run(["tar","-xzf",os.path.join(librispeech_dir ,"test_clean.tar.gz"),"-C",librispeech_dir])
    with tarfile.open(os.path.join(storage_dir , "test-clean.tar.gz"), "r:gz") as tar:
            tar.extractall(path=storage_dir )
            os.remove(os.path.join(storage_dir,"test-clean.tar.gz"))

In [ ]:
def wham():
    if not os.path.exists(wham_dir):
        print("Download wham_noise into", storage_dir)
        subprocess.run(['wget', '-c', '--tries=0', '--read-timeout=20', 'https://my-bucket-a8b4b49c25c811ee9a7e8bba05fa24c7.s3.amazonaws.com/wham_noise.zip', '-P', storage_dir])
        with zipfile.ZipFile(os.path.join(storage_dir, 'wham_noise.zip'), 'r') as zip_ref:
            zip_ref.extractall(storage_dir)
        os.remove(os.path.join(storage_dir, 'wham_noise.zip'))

In [ ]:
LibriSpeech_test_clean()

Download LibriSpeech/test_clean into /content


In [ ]:
wham()

Download wham_noise into /content


In [ ]:
# Global parameter
# We will filter out files shorter than that
NUMBER_OF_SECONDS = 3
# In LibriSpeech all the sources are at 16K Hz
RATE = 16000

def main(librispeech_dir):
    librispeech_dir = librispeech_dir
    # Create Librispeech metadata directory
    librispeech_md_dir = os.path.join(librispeech_dir, 'metadata')
    os.makedirs(librispeech_md_dir, exist_ok=True)
    create_librispeech_metadata(librispeech_dir, librispeech_md_dir)


def create_librispeech_metadata(librispeech_dir, md_dir):
    """ Generate metadata corresponding to downloaded data in LibriSpeech """
    # Get speakers metadata
    speakers_metadata = create_speakers_dataframe(librispeech_dir)
    # Check for already generated files and generate files accordingly
    dir_to_process = check_already_generated(md_dir, librispeech_dir)

    # Go through each directory and create associated metadata
    for ldir in dir_to_process:
        # Generate the dataframe relative to the directory
        dir_metadata = create_librispeech_dataframe(librispeech_dir, ldir,
                                                    speakers_metadata)
        # Filter out files that are shorter than 3s
        num_samples = NUMBER_OF_SECONDS * RATE
        dir_metadata = dir_metadata[
            dir_metadata['length'] >= num_samples]
        # Sort the dataframe according to ascending Length
        dir_metadata = dir_metadata.sort_values('length')
        # Write the dataframe in a .csv in the metadata directory
        save_path = os.path.join(md_dir, ldir + '.csv')
        print(dir_metadata.head())
        dir_metadata.to_csv(save_path, index=False)


def create_speakers_dataframe(librispeech_dir):
    """ Read metadata from the LibriSpeech dataset and collect infos
    about the speakers """
    print("Reading speakers metadata")
    # Read SPEAKERS.TXT and create a dataframe
    speakers_metadata_path = os.path.join(librispeech_dir, 'SPEAKERS.TXT')
    speakers_metadata = pd.read_csv(speakers_metadata_path, sep="|",
                                    skiprows=11,
                                    on_bad_lines='skip', header=0,
                                    names=['speaker_ID', 'sex', 'subset',
                                           'minutes', 'names'],
                                    skipinitialspace=True)
    # Drop useless columns
    speakers_metadata = speakers_metadata.drop(['minutes', 'names'], axis=1)
    # Delete white space
    for column in ['sex', 'subset']:
        speakers_metadata[column] = speakers_metadata[column].str.strip()
    # There is a problem with Speaker_ID = 60 his name contains " | " which is
    # the sperator character... Need to handle this case separately
    speakers_metadata.loc[len(speakers_metadata)] = [60, 'M',
                                                     'train-clean-100']

    return speakers_metadata


def check_already_generated(md_dir, librispeech_dir):
    # If md_dir already exists then check the already generated files
    already_generated_csv = os.listdir(md_dir)
    # Save the already generated files names
    already_generated_csv = [f.strip('.csv') for f in already_generated_csv]
    # Possible directories in the original LibriSpeech
    original_librispeech_dirs = ['dev-clean', 'dev-other', 'test-clean',
                                 'test-other', 'train-clean-100',
                                 'train-clean-360', 'train-other-500']
    # Actual directories extracted in your LibriSpeech version
    actual_librispeech_dirs = (set(next(os.walk(librispeech_dir))[1]) &
                               set(original_librispeech_dirs))
    # Actual directories that haven't already been processed
    not_already_processed_directories = list(set(actual_librispeech_dirs) -
                                             set(already_generated_csv))
    return not_already_processed_directories


def create_librispeech_dataframe(librispeech_dir, subdir, speakers_md):
    """ Generate a dataframe that gather infos about the sound files in a
    LibriSpeech subdirectory """

    print(f"Creating {subdir} metadata file in "
          f"{os.path.join(librispeech_dir, 'metadata')}")
    # Get the current directory path
    dir_path = os.path.join(librispeech_dir, subdir)
    # Recursively look for .flac files in current directory
    sound_paths = glob.glob(os.path.join(dir_path, '**/*.flac'),
                            recursive=True)
    # Create the dataframe corresponding to this directory
    dir_md = pd.DataFrame(columns=['speaker_ID', 'sex', 'subset',
                                   'length', 'origin_path'])

    # Go through the sound file list
    for sound_path in tqdm_module.tqdm(sound_paths, total=len(sound_paths)):
        # Get the ID of the speaker
        spk_id = os.path.split(sound_path)[1].split('-')[0]
        # Find Sex according to speaker ID in LibriSpeech metadata
        sex = speakers_md[speakers_md['speaker_ID'] == int(spk_id)].iat[0, 1]
        # Find subset according to speaker ID in LibriSpeech metadata
        subset = speakers_md[speakers_md['speaker_ID'] == int(spk_id)].iat[
            0, 2]
        # Get its length
        length = len(sf.SoundFile(sound_path))
        # Get the sound file relative path
        rel_path = os.path.relpath(sound_path, librispeech_dir)
        # Add information to the dataframe
        dir_md.loc[len(dir_md)] = [spk_id, sex, subset, length, rel_path]
    return dir_md


if __name__ == "__main__":
    main(librispeech_dir)

Reading speakers metadata
Creating test-clean metadata file in /content/LibriSpeech/metadata


100%|██████████| 2620/2620 [00:13<00:00, 195.62it/s]

     speaker_ID sex      subset  length  \
1588       8455   M  test-clean   48000   
116        3729   F  test-clean   48000   
1933       3575   F  test-clean   48000   
132        3729   F  test-clean   48000   
855        2094   F  test-clean   48080   

                                       origin_path  
1588  test-clean/8455/210777/8455-210777-0026.flac  
116       test-clean/3729/6852/3729-6852-0025.flac  
1933  test-clean/3575/170457/3575-170457-0052.flac  
132       test-clean/3729/6852/3729-6852-0027.flac  
855   test-clean/2094/142345/2094-142345-0045.flac  


In [ ]:
metadata = pd.read_csv('/content/LibriSpeech/metadata/test-clean.csv')
metadata.head()

,speaker_ID,sex,subset,length,origin_path
0,8455,M,test-clean,48000,test-clean/8455/210777/8455-210777-0026.flac
1,3729,F,test-clean,48000,test-clean/3729/6852/3729-6852-0025.flac
2,3575,F,test-clean,48000,test-clean/3575/170457/3575-170457-0052.flac
3,3729,F,test-clean,48000,test-clean/3729/6852/3729-6852-0027.flac
4,2094,F,test-clean,48080,test-clean/2094/142345/2094-142345-0045.flac


In [ ]:
# Global parameter
# We will filter out files shorter than that
NUMBER_OF_SECONDS = 3
# In WHAM! all the sources are at 16K Hz
RATE = 16000

# Command line arguments
wham_dir = os.path.join(storage_dir,"wham_noise")

def main():
    wham_noise_dir = wham_dir
    # Create wham_noise metadata directory
    wham_noise_md_dir = os.path.join(wham_noise_dir, 'metadata')
    os.makedirs(wham_noise_md_dir, exist_ok=True)
    create_wham_noise_metadata(wham_noise_dir, wham_noise_md_dir)


def create_wham_noise_metadata(wham_noise_dir, md_dir):
    """ Generate metadata corresponding to downloaded data in wham_noise """

    # Check already generated files
    not_already_processed_dir = check_already_generated(md_dir)
    # Go through each directory and create associated metadata
    for ldir in not_already_processed_dir:
        # Generate the dataframe relative to the directory
        dir_metadata = create_wham_noise_dataframe(wham_noise_dir, ldir)
        # Sort the dataframe according to ascending Length
        dir_metadata = dir_metadata.sort_values('length')
        # Write the dataframe in a .csv in the metadata directory
        if ldir == 'tt':
            name = 'test'
        elif ldir == 'cv':
            name = 'dev'
        else:
            name = 'train'
        # Filter out files that are shorter than 3s
        num_samples = NUMBER_OF_SECONDS * RATE
        dir_metadata = dir_metadata[
            dir_metadata['length'] >= num_samples]
        # Create save path
        save_path = os.path.join(md_dir, name + '.csv')
        print(f'Medatada file created in {save_path}')
        dir_metadata.to_csv(save_path, index=False)


def check_already_generated(md_dir):
    """ Check if files have already been generated """
    # Get the already generated files
    already_generated_csv = os.listdir(md_dir)
    # Data directories in wham_noise
    wham_noise_dirs = ['cv', 'tr', 'tt']
    # Save the already data directories names
    already_processed_dir = [
        f.replace("test", "tt").replace("train", "tr").replace("dev", "cv").
            replace(".csv", "") for f in already_generated_csv]
    # Actual directories that haven't already been processed
    not_already_processed_dir = list(set(wham_noise_dirs) -
                                     set(already_processed_dir))
    return not_already_processed_dir


def create_wham_noise_dataframe(wham_noise_dir, subdir):
    """ Generate a dataframe that gather infos about the sound files in a
    wham_noise subdirectory """

    print(f"Processing files from {subdir} dir")
    # Get the current directory path
    dir_path = os.path.join(wham_noise_dir, subdir)
    # Recursively look for .wav files in current directory
    sound_paths = glob.glob(os.path.join(dir_path, '**/*.wav'),
                            recursive=True)
    # Create the dataframe corresponding to this directory
    dir_md = pd.DataFrame(columns=['noise_ID', 'subset', 'length', 'augmented',
                                   'origin_path'])

    # Go through the sound file list
    for sound_path in tqdm_module.tqdm(sound_paths, total=len(sound_paths)):
        # Get the ID of the noise file
        noise_id = os.path.split(sound_path)[1]
        # Get its length
        length = len(sf.SoundFile(sound_path))
        augment = False
        if 'sp08' in sound_path or 'sp12' in sound_path:
            augment = True
        # Get the sound file relative path
        rel_path = os.path.relpath(sound_path, wham_noise_dir)
        # Add information to the dataframe
        dir_md.loc[len(dir_md)] = [noise_id, subdir, length, augment, rel_path]
    return dir_md


if __name__ == "__main__":
  main()

Processing files from tt dir


100%|██████████| 3000/3000 [00:12<00:00, 232.87it/s]


Medatada file created in /content/wham_noise/metadata/test.csv
Processing files from tr dir


100%|██████████| 20000/20000 [01:32<00:00, 217.05it/s]


Medatada file created in /content/wham_noise/metadata/train.csv
Processing files from cv dir


100%|██████████| 5000/5000 [00:18<00:00, 267.41it/s]

Medatada file created in /content/wham_noise/metadata/dev.csv


In [ ]:
# Global parameters
# eps secures log and division
EPS = 1e-10
# max amplitude in sources and mixtures
MAX_AMP = 0.9
# In LibriSpeech all the sources are at 16K Hz
RATE = 16000
# We will randomize loudness between this range
MIN_LOUDNESS = -33
MAX_LOUDNESS = -25

# A random seed is used for reproducibility
random.seed(72)

# Command line arguments

def main():
    storage_dir = "/content"
    librispeech_dir =  os.path.join(storage_dir,"LibriSpeech")
    librispeech_md_dir = "/content/LibriSpeech/metadata"
    metadata_outdir = os.path.join(storage_dir,"LibriMix")
    wham_dir = "/content/wham_noise"
    wham_md_dir = "/content/wham_noise/metadata"
    metadata_outdir =  "/content/LibriMix/metadata"
    n_src = 2
    md_dir = metadata_outdir
    if md_dir is None:
        root = os.path.dirname(storage_dir)
        md_dir = os.path.join(root, f'LibriMix/metadata')
    os.makedirs(md_dir, exist_ok=True)
    create_librimix_metadata(librispeech_dir, librispeech_md_dir, wham_dir, wham_md_dir, md_dir, n_src)


def create_librimix_metadata(librispeech_dir, librispeech_md_dir, wham_dir,
                             wham_md_dir, md_dir, n_src):
    """ Generate LibriMix metadata according to LibriSpeech metadata """

    # Dataset name
    dataset = f'libri{n_src}mix'
    # List metadata files in LibriSpeech
    librispeech_md_files = os.listdir(librispeech_md_dir)
    # List metadata files in wham_noise
    wham_md_files = os.listdir(wham_md_dir)
    # If you wish to ignore some metadata files add their name here
    # Example : to_be_ignored = ['dev-other.csv']
    to_be_ignored = []

    check_already_generated(md_dir, dataset, to_be_ignored,
                            librispeech_md_files)
    # Go through each metadata file and create metadata accordingly
    for librispeech_md_file in librispeech_md_files:
        if not librispeech_md_file.endswith('.csv'):
            print(f"{librispeech_md_file} is not a csv file, continue.")
            continue
        # Get the name of the corresponding noise md file
        try:
            wham_md_file = [f for f in wham_md_files if
                            f.startswith(librispeech_md_file.split('-')[0])][0]
        except IndexError:
            print('Wham metadata are missing you can either generate the '
                  'missing wham files or add the librispeech metadata to '
                  'to_be_ignored list')
            break

        # Open .csv files from LibriSpeech
        librispeech_md = pd.read_csv(os.path.join(
            librispeech_md_dir, librispeech_md_file), engine='python')
        # Open .csv files from wham_noise
        wham_md = pd.read_csv(os.path.join(
            wham_md_dir, wham_md_file), engine='python')
        # Filenames
        save_path = os.path.join(md_dir,
                                 '_'.join([dataset, librispeech_md_file]))
        info_name = '_'.join([dataset, librispeech_md_file.strip('.csv'),
                              'info']) + '.csv'
        info_save_path = os.path.join(md_dir, info_name)
        print(f"Creating {os.path.basename(save_path)} file in {md_dir}")
        # Create dataframe
        mixtures_md, mixtures_info = create_librimix_df(
            librispeech_md, librispeech_dir, wham_md, wham_dir,
            n_src)
        # Round number of files
        mixtures_md = mixtures_md[:len(mixtures_md) // 100 * 100]
        mixtures_info = mixtures_info[:len(mixtures_info) // 100 * 100]

        # Save csv files
        mixtures_md.to_csv(save_path, index=False)
        mixtures_info.to_csv(info_save_path, index=False)


def check_already_generated(md_dir, dataset, to_be_ignored,
                            librispeech_md_files):
    # Check if the metadata files in LibriSpeech already have been used
    already_generated = os.listdir(md_dir)
    for generated in already_generated:
        if generated.startswith(f"{dataset}") and 'info' not in generated:
            if 'train-100' in generated:
                to_be_ignored.append('train-clean-100.csv')
            elif 'train-360' in generated:
                to_be_ignored.append('train-clean-360.csv')
            elif 'dev' in generated:
                to_be_ignored.append('dev-clean.csv')
            elif 'test' in generated:
                to_be_ignored.append('test-clean.csv')
            print(f"{generated} already exists in "
                  f"{md_dir} it won't be overwritten")
    for element in to_be_ignored:
        librispeech_md_files.remove(element)


def create_librimix_df(librispeech_md_file, librispeech_dir,
                       wham_md_file, wham_dir, n_src):
    """ Generate librimix dataframe from a LibriSpeech and wha md file"""

    # Create a dataframe that will be used to generate sources and mixtures
    mixtures_md = pd.DataFrame(columns=['mixture_ID'])
    # Create a dataframe with additional infos.
    mixtures_info = pd.DataFrame(columns=['mixture_ID'])
    # Add columns (depends on the number of sources)
    for i in range(n_src):
        mixtures_md[f"source_{i + 1}_path"] = {}
        mixtures_md[f"source_{i + 1}_gain"] = {}
        mixtures_info[f"speaker_{i + 1}_ID"] = {}
        mixtures_info[f"speaker_{i + 1}_sex"] = {}
    mixtures_md["noise_path"] = {}
    mixtures_md["noise_gain"] = {}
    # Generate pairs of sources to mix
    pairs, pairs_noise = set_pairs(librispeech_md_file, wham_md_file, n_src)
    clip_counter = 0
    # For each combination create a new line in the dataframe
    for pair, pair_noise in tqdm_module.tqdm(zip(pairs, pairs_noise), total=len(pairs)):
        # return infos about the sources, generate sources
        sources_info, sources_list_max = read_sources(
            librispeech_md_file, pair, n_src, librispeech_dir)
        # Add noise
        sources_info, sources_list_max = add_noise(
            wham_md_file, wham_dir, pair_noise, sources_list_max, sources_info)
        # compute initial loudness, randomize loudness and normalize sources
        loudness, _, sources_list_norm = set_loudness(sources_list_max)
        # Do the mixture
        mixture_max = mix(sources_list_norm)
        # Check the mixture for clipping and renormalize if necessary
        renormalize_loudness, did_clip = check_for_cliping(mixture_max,
                                                           sources_list_norm)
        clip_counter += int(did_clip)
        # Compute gain
        gain_list = compute_gain(loudness, renormalize_loudness)

        # Add information to the dataframe
        row_mixture, row_info = get_row(sources_info, gain_list, n_src)
        mixtures_md.loc[len(mixtures_md)] = row_mixture
        mixtures_info.loc[len(mixtures_info)] = row_info
    print(f"Among {len(mixtures_md)} mixtures, {clip_counter} clipped.")
    return mixtures_md, mixtures_info


def set_pairs(librispeech_md_file, wham_md_file, n_src):
    """ set pairs of sources to make the mixture """
    # Initialize list for pairs sources
    utt_pairs = []
    noise_pairs = []
    # In train sets utterance are only used once
    if 'train' in librispeech_md_file.iloc[0]['subset']:
        utt_pairs = set_utt_pairs(librispeech_md_file, utt_pairs, n_src)
        noise_pairs = set_noise_pairs(utt_pairs, noise_pairs,
                                      librispeech_md_file, wham_md_file)
    # Otherwise we want 3000 mixtures
    else:
        while len(utt_pairs) < 3000:
            utt_pairs = set_utt_pairs(librispeech_md_file, utt_pairs, n_src)
            noise_pairs = set_noise_pairs(utt_pairs, noise_pairs,
                                          librispeech_md_file, wham_md_file)
            utt_pairs, noise_pairs = remove_duplicates(utt_pairs, noise_pairs)
        utt_pairs = utt_pairs[:3000]
        noise_pairs = noise_pairs[:3000]

    return utt_pairs, noise_pairs


def set_utt_pairs(librispeech_md_file, pair_list, n_src):
    # A counter
    c = 0
    # Index of the rows in the metadata file
    index = list(range(len(librispeech_md_file)))

    # Try to create pairs with different speakers end after 200 fails
    while len(index) >= n_src and c < 200:
        couple = random.sample(index, n_src)
        # Check that speakers are different
        speaker_list = set([librispeech_md_file.iloc[couple[i]]['speaker_ID']
                            for i in range(n_src)])
        # If there are duplicates then increment the counter
        if len(speaker_list) != n_src:
            c += 1
        # Else append the combination to pair_list and erase the combination
        # from the available indexes
        else:
            for i in range(n_src):
                index.remove(couple[i])
            pair_list.append(couple)
            c = 0
    return pair_list


def set_noise_pairs(pairs, noise_pairs, librispeech_md_file, wham_md_file):
    print('Generating pairs')
    # Initially take not augmented data
    md = wham_md_file[wham_md_file['augmented'] == False]
    # If there are more mixtures than noises then use augmented data
    if len(pairs) > len(md):
        md = wham_md_file
    # Copy pairs because we are going to remove elements from pairs
    for pair in pairs.copy():
        # get sources infos
        sources = [librispeech_md_file.iloc[pair[i]]
                   for i in range(len(pair))]
        # get max_length
        length_list = [source['length'] for source in sources]
        max_length = max(length_list)
        # Ideal choices are noises longer than max_length
        possible = md[md['length'] >= max_length]
        # if possible is not empty
        try:
            # random noise longer than max_length
            pair_noise = random.sample(list(possible.index), 1)
            # add that noise's index to the list
            noise_pairs.append(pair_noise)
            # remove that noise from the remaining noises
            md = md.drop(pair_noise)
        # if possible is empty
        except ValueError:
            # if we deal with training files
            if 'train' in librispeech_md_file.iloc[0]['subset']:
                # take the longest noise remaining
                pair_noise = list(md.index)[-1]
                # add it to noise list
                noise_pairs.append(pair_noise)
                # remove it from remaining noises
                md = md.drop(pair_noise)
            # if dev or test
            else:
                # just delete the pair we will redo this process
                pairs.remove(pair)

    return noise_pairs


def remove_duplicates(utt_pairs, noise_pairs):
    print('Removing duplicates')
    # look for identical mixtures O(n²)
    for i, (pair, pair_noise) in enumerate(zip(utt_pairs, noise_pairs)):
        for j, (du_pair, du_pair_noise) in enumerate(
                zip(utt_pairs, noise_pairs)):
            # sort because [s1,s2] = [s2,s1]
            if sorted(pair) == sorted(du_pair) and i != j:
                utt_pairs.remove(du_pair)
                noise_pairs.remove(du_pair_noise)
    return utt_pairs, noise_pairs


def read_sources(metadata_file, pair, n_src, librispeech_dir):
    # Read lines corresponding to pair
    sources = [metadata_file.iloc[pair[i]] for i in range(n_src)]
    # Get sources info
    speaker_id_list = [source['speaker_ID'] for source in sources]
    sex_list = [source['sex'] for source in sources]
    length_list = [source['length'] for source in sources]
    path_list = [source['origin_path'] for source in sources]
    id_l = [os.path.split(source['origin_path'])[1].strip('.flac')
            for source in sources]
    mixtures_id = "_".join(id_l)

    # Get the longest and shortest source len
    max_length = max(length_list)
    sources_list = []

    # Read the source and compute some info
    for i in range(n_src):
        source = metadata_file.iloc[pair[i]]
        absolute_path = os.path.join(librispeech_dir,
                                     source['origin_path'])
        s, _ = sf.read(absolute_path, dtype='float32')
        sources_list.append(
            np.pad(s, (0, max_length - len(s)), mode='constant'))

    sources_info = {'mixtures_id': mixtures_id,
                    'speaker_id_list': speaker_id_list, 'sex_list': sex_list,
                    'path_list': path_list}
    return sources_info, sources_list


def add_noise(wham_md_file, wham_dir, pair_noise, sources_list, sources_info):
    # Get the row corresponding to the index
    noise = wham_md_file.loc[pair_noise]
    # Get the noise path
    try:
        noise_path = os.path.join(wham_dir, noise['origin_path'].values[0])
    except AttributeError:
        noise_path = os.path.join(wham_dir, noise['origin_path'])
    # Read the noise
    n, _ = sf.read(noise_path, dtype='float32')
    # Keep the first channel
    if len(n.shape) > 1:
        n = n[:, 0]
    # Get expected length
    length = len(sources_list[0])
    # Pad if shorter
    if length > len(n):
        sources_list.append(np.pad(n, (0, length - len(n)), mode='constant'))
    # Cut if longer
    else:
        sources_list.append(n[:length])
    # Get relative path
    try :
        sources_info['noise_path'] = noise['origin_path'].values[0]
    except AttributeError:
        sources_info['noise_path'] = noise['origin_path']
    return sources_info, sources_list


def set_loudness(sources_list):
    """ Compute original loudness and normalise them randomly """
    # Initialize loudness
    loudness_list = []
    # In LibriSpeech all sources are at 16KHz hence the meter
    meter = pyln.Meter(RATE)
    # Randomize sources loudness
    target_loudness_list = []
    sources_list_norm = []

    # Normalize loudness
    for i in range(len(sources_list)):
        # Compute initial loudness
        loudness_list.append(meter.integrated_loudness(sources_list[i]))
        # Pick a random loudness
        target_loudness = random.uniform(MIN_LOUDNESS, MAX_LOUDNESS)
        # Noise has a different loudness
        if i == len(sources_list) - 1:
            target_loudness = random.uniform(MIN_LOUDNESS - 5,
                                             MAX_LOUDNESS - 5)
        # Normalize source to target loudness

        with warnings.catch_warnings():
            # We don't want to pollute stdout, but we don't want to ignore
            # other warnings.
            warnings.simplefilter("ignore")
            src = pyln.normalize.loudness(sources_list[i], loudness_list[i],
                                          target_loudness)
        # If source clips, renormalize
        if np.max(np.abs(src)) >= 1:
            src = sources_list[i] * MAX_AMP / np.max(np.abs(sources_list[i]))
            target_loudness = meter.integrated_loudness(src)
        # Save scaled source and loudness.
        sources_list_norm.append(src)
        target_loudness_list.append(target_loudness)
    return loudness_list, target_loudness_list, sources_list_norm


def mix(sources_list_norm):
    """ Do the mixture for min mode and max mode """
    # Initialize mixture
    mixture_max = np.zeros_like(sources_list_norm[0])
    for i in range(len(sources_list_norm)):
        mixture_max += sources_list_norm[i]
    return mixture_max


def check_for_cliping(mixture_max, sources_list_norm):
    """Check the mixture (mode max) for clipping and re normalize if needed."""
    # Initialize renormalized sources and loudness
    renormalize_loudness = []
    clip = False
    # Recreate the meter
    meter = pyln.Meter(RATE)
    # Check for clipping in mixtures
    if np.max(np.abs(mixture_max)) > MAX_AMP:
        clip = True
        weight = MAX_AMP / np.max(np.abs(mixture_max))
    else:
        weight = 1
    # Renormalize
    for i in range(len(sources_list_norm)):
        new_loudness = meter.integrated_loudness(sources_list_norm[i] * weight)
        renormalize_loudness.append(new_loudness)
    return renormalize_loudness, clip


def compute_gain(loudness, renormalize_loudness):
    """ Compute the gain between the original and target loudness"""
    gain = []
    for i in range(len(loudness)):
        delta_loudness = renormalize_loudness[i] - loudness[i]
        gain.append(np.power(10.0, delta_loudness / 20.0))
    return gain


def get_row(sources_info, gain_list, n_src):
    """ Get new row for each mixture/info dataframe """
    row_mixture = [sources_info['mixtures_id']]
    row_info = [sources_info['mixtures_id']]
    for i in range(n_src):
        row_mixture.append(sources_info['path_list'][i])
        row_mixture.append(gain_list[i])
        row_info.append(sources_info['speaker_id_list'][i])
        row_info.append(sources_info['sex_list'][i])
    row_mixture.append(sources_info['noise_path'])
    row_mixture.append(gain_list[-1])
    return row_mixture, row_info


if __name__ == "__main__":
  main()

Creating libri2mix_test-clean.csv file in /content/LibriMix/metadata
Generating pairs
Removing duplicates
Generating pairs
Removing duplicates
Generating pairs
Removing duplicates
Generating pairs
Removing duplicates
Generating pairs
Removing duplicates


100%|██████████| 3000/3000 [03:48<00:00, 13.13it/s]


Among 3000 mixtures, 7 clipped.


In [ ]:
librimix_metadata = pd.read_csv("/content/LibriMix/metadata/libri2mix_test-clean.csv")
librimix_metadata.head()

,mixture_ID,source_1_path,source_1_gain,source_2_path,source_2_gain,noise_path,noise_gain
0,3575-170457-0025_1221-135766-0004,test-clean/3575/170457/3575-170457-0025.flac,0.475709,test-clean/1221/135766/1221-135766-0004.flac,1.029462,tt/050o020x_2.2523_444c020n_-2.2523.wav,0.351628
1,1580-141084-0034_1995-1837-0012,test-clean/1580/141084/1580-141084-0034.flac,1.017748,test-clean/1995/1837/1995-1837-0012.flac,0.529042,tt/440c0201_2.437_423o030s_-2.437.wav,0.586591
2,8463-294828-0020_3570-5694-0008,test-clean/8463/294828/8463-294828-0020.flac,0.553068,test-clean/3570/5694/3570-5694-0008.flac,0.523287,tt/445c020t_1.3052_053a050q_-1.3052.wav,0.847530
3,1580-141084-0009_2830-3980-0022,test-clean/1580/141084/1580-141084-0009.flac,0.569410,test-clean/2830/3980/2830-3980-0022.flac,0.695181,tt/422c0215_0.49701_445c0216_-0.49701.wav,1.981860
4,3729-6852-0038_7021-79759-0005,test-clean/3729/6852/3729-6852-0038.flac,0.471777,test-clean/7021/79759/7021-79759-0005.flac,0.546788,tt/440c020c_1.7835_420a010z_-1.7835.wav,1.617649


In [ ]:
import os
import argparse
import soundfile as sf
import pandas as pd
import numpy as np
import functools
from scipy.signal import resample_poly
import tqdm.contrib.concurrent

# eps secures log and division
EPS = 1e-10
# Rate of the sources in LibriSpeech
RATE = 16000

def main():

    librispeech_dir = "/content/LibriSpeech"

    wham_dir = "/content/wham_noise"

    metadata_dir = "/content/LibriMix/metadata"

    librimix_outdir = "/content/LibriMix"

    n_src = 2

    freqs = ['8k']

    modes = ['max']

    types = ['mix_both']

    freqs = [freq.lower() for freq in freqs]

    modes = [mode.lower() for mode in modes]

    types = [t.lower() for t in types]

    # Get the number of sources
    create_librimix(librispeech_dir, wham_dir, librimix_outdir, metadata_dir, freqs, n_src, modes, types)


def create_librimix(librispeech_dir, wham_dir, out_dir, metadata_dir,
                    freqs, n_src, modes, types):
    """ Generate sources mixtures and saves them in out_dir"""
    # Get metadata files
    md_filename_list = [file for file in os.listdir(metadata_dir)
                        if 'info' not in file]
    # Create all parts of librimix
    for md_filename in md_filename_list:
        csv_path = os.path.join(metadata_dir, md_filename)
        process_metadata_file(csv_path, freqs, n_src, librispeech_dir,
                              wham_dir, out_dir, modes, types)


def process_metadata_file(csv_path, freqs, n_src, librispeech_dir, wham_dir,
                          out_dir, modes, types):
    """ Process a metadata generation file to create sources and mixtures"""
    md_file = pd.read_csv(csv_path, engine='python')
    for freq in freqs:
        # Get the frequency directory path
        freq_path = os.path.join(out_dir, 'wav' + freq)
        # Transform freq = "16k" into 16000
        freq = int(freq.strip('k')) * 1000

        for mode in modes:
            # Path to the mode directory
            mode_path = os.path.join(freq_path, mode)
            # Subset metadata path
            subset_metadata_path = os.path.join(mode_path, 'metadata')
            os.makedirs(subset_metadata_path, exist_ok=True)
            # Directory where the mixtures and sources will be stored
            dir_name = os.path.basename(csv_path).replace(
                f'libri{n_src}mix_', '').replace('-clean', '').replace(
                '.csv', '')
            dir_path = os.path.join(mode_path, dir_name)
            # If the files already exist then continue the loop
            if os.path.isdir(dir_path):
                print(f"Directory {dir_path} already exist. "
                      f"Files won't be overwritten")
                continue

            print(f"Creating mixtures and sources from {csv_path} "
                  f"in {dir_path}")
            # Create subdir
            if types == ['mix_clean']:
                subdirs = [f's{i + 1}' for i in range(n_src)] + ['mix_clean']
            else:
                subdirs = [f's{i + 1}' for i in range(n_src)] + types + [
                    'noise']
            # Create directories accordingly
            for subdir in subdirs:
                os.makedirs(os.path.join(dir_path, subdir))
            # Go through the metadata file
            process_utterances(md_file, librispeech_dir, wham_dir, freq, mode,
                               subdirs, dir_path, subset_metadata_path, n_src)


def process_utterances(md_file, librispeech_dir, wham_dir, freq, mode, subdirs,
                       dir_path, subset_metadata_path, n_src):
    # Dictionary that will contain all metadata
    md_dic = {}
    # Get dir name
    dir_name = os.path.basename(dir_path)
    # Create Dataframes
    for subdir in subdirs:
        if subdir.startswith('mix'):
            md_dic[f'metrics_{dir_name}_{subdir}'] = create_empty_metrics_md(
                n_src, subdir)
            md_dic[f'mixture_{dir_name}_{subdir}'] = create_empty_mixture_md(
                n_src, subdir)

    # Go through the metadata file and generate mixtures
    for results in tqdm_module.contrib.concurrent.process_map(
        functools.partial(
            process_utterance,
            n_src, librispeech_dir, wham_dir, freq, mode, subdirs, dir_path),
        [row for _, row in md_file.iterrows()],
        chunksize=10,
    ):
        for mix_id, snr_list, abs_mix_path, abs_source_path_list, abs_noise_path, length, subdir in results:
            # Add line to the dataframes
            add_to_metrics_metadata(md_dic[f"metrics_{dir_name}_{subdir}"],
                                    mix_id, snr_list)
            add_to_mixture_metadata(md_dic[f'mixture_{dir_name}_{subdir}'],
                                    mix_id, abs_mix_path, abs_source_path_list,
                                    abs_noise_path, length, subdir)

    # Save the metadata files
    for md_df in md_dic:
        # Save the metadata in out_dir ./data/wavxk/mode/subset
        save_path_mixture = os.path.join(subset_metadata_path, md_df + '.csv')
        md_dic[md_df].to_csv(save_path_mixture, index=False)


def process_utterance(n_src, librispeech_dir, wham_dir, freq, mode, subdirs, dir_path, row):
    res = []
    # Get sources and mixture infos
    mix_id, gain_list, sources = read_sources(row, n_src, librispeech_dir,
                                              wham_dir)
    # Transform sources
    transformed_sources = transform_sources(sources, freq, mode, gain_list)
    # Write the sources and get their paths
    abs_source_path_list = write_sources(mix_id,
                                         transformed_sources,
                                         subdirs, dir_path, freq,
                                         n_src)
    # Write the noise and get its path
    abs_noise_path = write_noise(mix_id, transformed_sources, dir_path,
                                 freq)
    # Mixtures are different depending on the subdir
    for subdir in subdirs:
        if subdir == 'mix_clean':
            sources_to_mix = transformed_sources[:n_src]
        elif subdir == 'mix_both':
            sources_to_mix = transformed_sources
        elif subdir == 'mix_single':
            sources_to_mix = [transformed_sources[0],
                              transformed_sources[-1]]
        else:
            continue

        # Mix sources
        mixture = mix(sources_to_mix)
        # Write mixture and get its path
        abs_mix_path = write_mix(mix_id, mixture, dir_path, subdir, freq)
        length = len(mixture)
        # Compute SNR
        snr_list = compute_snr_list(mixture, sources_to_mix)
        res.append((mix_id, snr_list, abs_mix_path, abs_source_path_list, abs_noise_path, length, subdir))

    return res


def create_empty_metrics_md(n_src, subdir):
    """ Create the metrics dataframe"""
    metrics_dataframe = pd.DataFrame()
    metrics_dataframe['mixture_ID'] = {}
    if subdir == 'mix_clean':
        for i in range(n_src):
            metrics_dataframe[f"source_{i + 1}_SNR"] = {}
    elif subdir == 'mix_both':
        for i in range(n_src):
            metrics_dataframe[f"source_{i + 1}_SNR"] = {}
        metrics_dataframe[f"noise_SNR"] = {}
    elif subdir == 'mix_single':
        metrics_dataframe["source_1_SNR"] = {}
        metrics_dataframe[f"noise_SNR"] = {}
    return metrics_dataframe


def create_empty_mixture_md(n_src, subdir):
    """ Create the mixture dataframe"""
    mixture_dataframe = pd.DataFrame()
    mixture_dataframe['mixture_ID'] = {}
    mixture_dataframe['mixture_path'] = {}
    if subdir == 'mix_clean':
        for i in range(n_src):
            mixture_dataframe[f"source_{i + 1}_path"] = {}
    elif subdir == 'mix_both':
        for i in range(n_src):
            mixture_dataframe[f"source_{i + 1}_path"] = {}
        mixture_dataframe[f"noise_path"] = {}
    elif subdir == 'mix_single':
        mixture_dataframe["source_1_path"] = {}
        mixture_dataframe[f"noise_path"] = {}
    mixture_dataframe['length'] = {}
    return mixture_dataframe


def read_sources(row, n_src, librispeech_dir, wham_dir):
    """ Get sources and info to mix the sources """
    # Get info about the mixture
    mixture_id = row['mixture_ID']
    sources_path_list = get_list_from_csv(row, 'source_path', n_src)
    gain_list = get_list_from_csv(row, 'source_gain', n_src)
    sources_list = []
    max_length = 0
    # Read the files to make the mixture
    for sources_path in sources_path_list:
        sources_path = os.path.join(librispeech_dir,
                                    sources_path)
        source, _ = sf.read(sources_path, dtype='float32')
        # Get max_length
        if max_length < len(source):
            max_length = len(source)
        sources_list.append(source)
    # Read the noise
    noise_path = os.path.join(wham_dir, row['noise_path'])
    noise, _ = sf.read(noise_path, dtype='float32', stop=max_length)
    # if noises have 2 channels take the first
    if len(noise.shape) > 1:
        noise = noise[:, 0]
    # if noise is too short extend it
    if len(noise) < max_length:
        noise = extend_noise(noise, max_length)
    sources_list.append(noise)
    gain_list.append(row['noise_gain'])

    return mixture_id, gain_list, sources_list


def get_list_from_csv(row, column, n_src):
    """ Transform a list in the .csv in an actual python list """
    python_list = []
    for i in range(n_src):
        current_column = column.split('_')
        current_column.insert(1, str(i + 1))
        current_column = '_'.join(current_column)
        python_list.append(row[current_column])
    return python_list


def extend_noise(noise, max_length):
    """ Concatenate noise using hanning window"""
    noise_ex = noise
    window = np.hanning(RATE + 1)
    # Increasing window
    i_w = window[:len(window) // 2 + 1]
    # Decreasing window
    d_w = window[len(window) // 2::-1]
    # Extend until max_length is reached
    while len(noise_ex) < max_length:
        noise_ex = np.concatenate((noise_ex[:len(noise_ex) - len(d_w)],
                                   np.multiply(
                                       noise_ex[len(noise_ex) - len(d_w):],
                                       d_w) + np.multiply(
                                       noise[:len(i_w)], i_w),
                                   noise[len(i_w):]))
    noise_ex = noise_ex[:max_length]
    return noise_ex


def transform_sources(sources_list, freq, mode, gain_list):
    """ Transform libriSpeech sources to librimix """
    # Normalize sources
    sources_list_norm = loudness_normalize(sources_list, gain_list)
    # Resample the sources
    sources_list_resampled = resample_list(sources_list_norm, freq)
    # Reshape sources
    reshaped_sources = fit_lengths(sources_list_resampled, mode)
    return reshaped_sources


def loudness_normalize(sources_list, gain_list):
    """ Normalize sources loudness"""
    # Create the list of normalized sources
    normalized_list = []
    for i, source in enumerate(sources_list):
        normalized_list.append(source * gain_list[i])
    return normalized_list


def resample_list(sources_list, freq):
    """ Resample the source list to the desired frequency"""
    # Create the resampled list
    resampled_list = []
    # Resample each source
    for source in sources_list:
        resampled_list.append(resample_poly(source, freq, RATE))
    return resampled_list


def fit_lengths(source_list, mode):
    """ Make the sources to match the target length """
    sources_list_reshaped = []
    # Check the mode
    if mode == 'min':
        target_length = min([len(source) for source in source_list])
        for source in source_list:
            sources_list_reshaped.append(source[:target_length])
    else:
        target_length = max([len(source) for source in source_list])
        for source in source_list:
            sources_list_reshaped.append(
                np.pad(source, (0, target_length - len(source)),
                       mode='constant'))
    return sources_list_reshaped


def write_sources(mix_id, transformed_sources, subdirs, dir_path, freq, n_src):
    # Write sources and mixtures and save their path
    abs_source_path_list = []
    ex_filename = mix_id + '.wav'
    for src, src_dir in zip(transformed_sources[:n_src], subdirs[:n_src]):
        save_path = os.path.join(dir_path, src_dir, ex_filename)
        abs_save_path = os.path.abspath(save_path)
        sf.write(abs_save_path, src, freq)
        abs_source_path_list.append(abs_save_path)
    return abs_source_path_list


def write_noise(mix_id, transformed_sources, dir_path, freq):
    # Write noise save it's path
    noise = transformed_sources[-1]
    ex_filename = mix_id + '.wav'
    save_path = os.path.join(dir_path, 'noise', ex_filename)
    abs_save_path = os.path.abspath(save_path)
    sf.write(abs_save_path, noise, freq)
    return abs_save_path


def mix(sources_list):
    """ Do the mixing """
    # Initialize mixture
    mixture = np.zeros_like(sources_list[0])
    for source in sources_list:
        mixture += source
    return mixture


def write_mix(mix_id, mixture, dir_path, subdir, freq):
    # Write noise save it's path
    ex_filename = mix_id + '.wav'
    save_path = os.path.join(dir_path, subdir, ex_filename)
    abs_save_path = os.path.abspath(save_path)
    sf.write(abs_save_path, mixture, freq)
    return abs_save_path


def compute_snr_list(mixture, sources_list):
    """Compute the SNR on the mixture mode min"""
    snr_list = []
    # Compute SNR for min mode
    for i in range(len(sources_list)):
        noise_min = mixture - sources_list[i]
        snr_list.append(snr_xy(sources_list[i], noise_min))
    return snr_list


def snr_xy(x, y):
    return 10 * np.log10(np.mean(x ** 2) / (np.mean(y ** 2) + EPS) + EPS)


def add_to_metrics_metadata(metrics_df, mixture_id, snr_list):
    """ Add a new line to metrics_df"""
    row_metrics = [mixture_id] + snr_list
    metrics_df.loc[len(metrics_df)] = row_metrics


def add_to_mixture_metadata(mix_df, mix_id, abs_mix_path, abs_sources_path,
                            abs_noise_path, length, subdir):
    """ Add a new line to mixture_df """
    sources_path = abs_sources_path
    noise_path = [abs_noise_path]
    if subdir == 'mix_clean':
        noise_path = []
    elif subdir == 'mix_single':
        sources_path = [abs_sources_path[0]]
    row_mixture = [mix_id, abs_mix_path] + sources_path + noise_path + [length]
    mix_df.loc[len(mix_df)] = row_mixture


if __name__ == "__main__":
    main()

Creating mixtures and sources from /content/LibriMix/metadata/libri2mix_test-clean.csv in /content/LibriMix/wav8k/max/test


  0%|          | 0/3000 [00:00<?, ?it/s]

In [ ]:
# import shutil
# from google.colab import files

# # Example: Compress a folder into a zip file
# shutil.make_archive('/LibriMix', 'zip', '/content/LibriMix')

'/LibriMix.zip'

In [ ]:
# # Download the compressed file
# files.download('/LibriMix.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [5]:
class LibriMixDataset(Dataset):
    def __init__(self, mix_files, s1_dir, s2_dir, transform=None):
        self.mix_both_dir = mix_both_dir
        self.s1_dir = s1_dir
        self.s2_dir = s2_dir
        self.mix_files = mix_files
        self.transform = transform

    def __len__(self):
        return len(self.mix_files)

    def __getitem__(self, idx):
        mix_file = self.mix_files[idx]
        mix_path = os.path.join(self.mix_both_dir, mix_file)
        s1_path = os.path.join(self.s1_dir, mix_file)
        s2_path = os.path.join(self.s2_dir, mix_file)

        # Load mixed audio and sources
        mix_audio, _ = torchaudio.load(mix_path)
        s1_audio, _ = torchaudio.load(s1_path)
        s2_audio, _ = torchaudio.load(s2_path)

        return {'mixed_audio': mix_audio, 'source_1': s1_audio, 'source_2': s2_audio,'mixed_audio_path':mix_path}

# Create collate function to pad each audio to the maximum audio size of the whole batch
def collate_fn(batch):
    # Get maximum length among all audio files in the batch
    max_len = max(max(sample['mixed_audio'].size(1), sample['source_1'].size(1), sample['source_2'].size(1)) for sample in batch)

    # Pad each audio file in the batch to match the maximum length
    padded_batch = []
    for sample in batch:
        padded_mixed_audio = torch.nn.functional.pad(sample['mixed_audio'], (0, max_len - sample['mixed_audio'].size(1)))
        padded_source_1 = torch.nn.functional.pad(sample['source_1'], (0, max_len - sample['source_1'].size(1)))
        padded_source_2 = torch.nn.functional.pad(sample['source_2'], (0, max_len - sample['source_2'].size(1)))
        padded_batch.append({'mixed_audio': padded_mixed_audio, 'source_1': padded_source_1, 'source_2': padded_source_2 , 'mixed_audio_path':sample['mixed_audio_path']})

    return padded_batch

mix_both_dir = "/content/LibriMix/wav8k/max/test/mix_both"
s1_dir = " /content/LibriMix/wav8k/max/test/s1"
s2_dir = " /content/LibriMix/wav8k/max/test/s2"



file_names = os.listdir(mix_both_dir)

split_idx = int(0.7 * len(file_names))

train_file_names = file_names[:split_idx]
test_file_names = file_names[split_idx:]

# Create datasets
train_dataset = LibriMixDataset(train_file_names, s1_dir, s2_dir)
test_dataset = LibriMixDataset(test_file_names, s1_dir, s2_dir)

# dataset = LibriMixDataset(mix_both_dir, s1_dir, s2_dir)
test_dataloader = DataLoader(test_dataset, batch_size=4, collate_fn=collate_fn)

In [6]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available. Using GPU.")
else:
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU.")

CUDA is available. Using GPU.


In [26]:
def evaluate_model(model,dataloader,device):
  model.eval()
  sdr_metric = ScaleInvariantSignalDistortionRatio()
  sisnri_metric = ScaleInvariantSignalNoiseRatio()
  with torch.no_grad():
    for batch in tqdm_module.tqdm(dataloader):
      for sample in batch:
        mixed_audio , source_1 , source_2 = sample['mixed_audio'] , sample['source_1'] , sample['source_2']
        source_1 = source_1.to(device)
        source_2 = source_2.to(device)
        mixed_audio = mixed_audio.to(device)
        model=model.to('cuda')
        # print(model.device)
        est_sources = model(mixed_audio)
        sdr_metric.update(est_sources,torch.stack((source_1,source_2),dim=2))
        sisnri_metric.update(est_sources,torch.stack((source_1,source_2),dim=2))

    sdr_score = sdr_metric.compute()
    sisnri_score = sisnri_metric.compute()
    return sdr_score,sisnri_score

In [2]:
model = separator.from_hparams(source="speechbrain/sepformer-whamr", savedir='pretrained_models/sepformer-whamr')

model = model.to("cuda")

print(model.device)

sdr_score, sisnri_score = evaluate_model(model,test_dataloader,device)

print(f"SDR Score: {sdr_score}")
print(f"SISNRi Score: {sisnri_score}")

In [ ]:
# audio,_ = torchaudio.load('/content/wav8k/max/test/mix_both/1089-134686-0000_121-127105-0013.wav')

In [ ]:
# audio = audio.to(device)

In [3]:
# audio.device

In [ ]:
# audio = audio.to('cpu')

In [ ]:
# model = model.to('cpu')

In [ ]:
# est_sources = model(audio)

In [1]:
# Audio(est_sources[:, :, 0].detach().cpu().squeeze(), rate=8000)

In [ ]:
# Audio(est_sources[:, :, 1].detach().cpu().squeeze(), rate=8000)